In [1]:
%load_ext pycodestyle_magic
%load_ext jupyternotify
%load_ext autoreload
%autoreload 2
%matplotlib inline

<IPython.core.display.Javascript object>

In [2]:
import sys
sys.path.append('../')

In [ ]:
import torch

In [3]:
from utils.data import MNIST_dataset

In [ ]:
train_loader = MNIST_dataset(
    train=True, cuda=False
).get_loader()

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]